# Evaluating quality resutls

In [1]:
import geopandas as gpd
import folium
import json
from tqdm import tqdm
import re

from custom_functions.gtfs_methods import detect_direction, match_schedule_for_service_line
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)

In the theoretical schedule, we are supposed to have 5 686 375 rows

In [2]:
th = pd.read_pickle("data/computed/schedule_normalized_dist.pkl")
th.shape

(5686375, 53)

4 177 334 rows where effective matches and not empty fields, which is about ~73% of the vehicules that could have been detected.

In [3]:
res = pd.read_pickle("data/computed/finally_matches.pkl")
res

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,route_id,service_id,trip_headsign,direction_id,block_id,shape_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,time_seconds,headway_min,clusters,cluster_agg_value,regularity,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,start_date_ft,end_date_ft,label,date_normalized,stop_id_clean_c,descr_fr,descr_nl,geometry,geom_position_before,dist_m,date,time,distance_from_point,stop_id__current,stop_id_cleaned,stop_id__terminus,stop_code__terminus,stop_name__terminus,stop_desc__terminus,stop_lat__terminus,stop_lon__terminus,zone_id__terminus,stop_url__terminus,location_type__terminus,parent_station__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,lag_-1,lag_+1,t_lag_before,t_diff(sec),adjusted_arrival_time(ts),adjusted_arrival_time,diff_time,vehicule_id,approx_rec
0,112947202236269500,07:30:56,07:30:56,8733,1,0,0,NaN,GARE DE L'OUEST,NaN,50.848999,4.320948,NaN,NaN,0.0,2.0,2,236269500,STOCKEL,0,8912798,001m0080,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF,26738,15.0,cluster_0,15.0,0,0,0,0,0,0,1,0,20210904,20210918,2021-09-04,2021-09-18,saturday,11-09-2021,8733,GARE DE L'OUEST,WESTSTATION,POINT (146633.5 170956.4),None,NaN,11-09-2021,07:25:38,0.0,8733,8733,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161,saturday,1,M,0.0,0.0,26151.0,587.0,26738.000,07:25:38,318.000,0,NaN
1,112947202236269500,07:31:53,07:31:53,8742,2,0,0,NaN,BEEKKANT,NaN,50.853386,4.322974,NaN,NaN,0.0,1.0,2,236269500,STOCKEL,0,8912798,001m0080,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF,27141,15.0,cluster_0,15.0,0,0,0,0,0,0,1,0,20210904,20210918,2021-09-04,2021-09-18,saturday,11-09-2021,8742,BEEKKANT,BEEKKANT,POINT (146776.5 171444.3),POINT (146633.5 170956.4),508.424439,11-09-2021,07:32:21,0.0,8742,8742,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161,saturday,1,M,1.0,1.0,26308.0,833.0,27141.000,07:32:21,28.000,0,NaN
2,112947202236269500,07:33:33,07:33:33,8292,3,0,0,NaN,ETANGS NOIRS,NaN,50.857125,4.333143,NaN,NaN,0.0,12.0,2,236269500,STOCKEL,0,8912798,001m0080,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF,27233,15.0,cluster_0,15.0,0,0,0,0,0,0,1,0,20210904,20210918,2021-09-04,2021-09-18,saturday,11-09-2021,8292,ETANGS NOIRS,ZWARTE VIJVERS,POINT (147492.7 171859.9),POINT (146776.5 171444.3),828.049395,11-09-2021,07:33:53,0.0,8292,8292,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161,saturday,1,M,1.0,1.0,26370.0,863.0,27233.000,07:33:53,20.000,0,NaN
3,112947202236269500,07:34:44,07:34:44,8282,4,0,0,NaN,COMTE DE FLANDRE,NaN,50.854705,4.340542,NaN,NaN,0.0,13.0,2,236269500,STOCKEL,0,8912798,001m0080,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF,27295,15.0,cluster_0,15.0,0,0,0,0,0,0,1,0,20210904,20210918,2021-09-04,2021-09-18,saturday,11-09-2021,8282,COMTE DE FLANDRE,GRAAF VAN VLAAND.,POINT (148013.6 171590.4),POINT (147492.7 171859.9),586.487050,11-09-2021,07:34:55,0.0,8282,8282,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161,saturday,1,M,1.0,1.0,26432.0,863.0,27295.000,07:34:55,11.000,0,NaN
4,112947202236269500,07:36:08,07:36:08,8272,5,0,0,NaN,SAINTE-CATHERINE,NaN,50.851900,4.348012,NaN,NaN,0.0,14.0,2,236269500,STOCKEL,0,8912798,001m0080,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF,27385,15.0,cluster_0,15.0,0,0,0,0,0,0,1,0,20210904,20210918,2021-09-04,2021-09-18,saturday,11-09-2021,8272,SAINTE-CATHERINE,SINT-KATELIJNE,POINT (148539.5 171278.2),POINT (148013.6 171590.4),611.587811,11-09-2021,07:36:25,0.0,8272,8272,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161,saturday,1,M,1.0,1.0,26524.0,861.0,27385.000,07:36:25,17.000,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [4]:
res[-res.adjusted_arrival_time.isna()]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,route_id,service_id,trip_headsign,direction_id,block_id,shape_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,time_seconds,headway_min,clusters,cluster_agg_value,regularity,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,start_date_ft,end_date_ft,label,date_normalized,stop_id_clean_c,descr_fr,descr_nl,geometry,geom_position_before,dist_m,date,time,distance_from_point,stop_id__current,stop_id_cleaned,stop_id__terminus,stop_code__terminus,stop_name__terminus,stop_desc__terminus,stop_lat__terminus,stop_lon__terminus,zone_id__terminus,stop_url__terminus,location_type__terminus,parent_station__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,lag_-1,lag_+1,t_lag_before,t_diff(sec),adjusted_arrival_time(ts),adjusted_arrival_time,diff_time,vehicule_id,approx_rec
0,112947202236269500,07:30:56,07:30:56,8733,1,0,0,NaN,GARE DE L'OUEST,NaN,50.848999,4.320948,NaN,NaN,0.0,2.0,2,236269500,STOCKEL,0,8912798,001m0080,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF,26738,15.0,cluster_0,15.0,0,0,0,0,0,0,1,0,20210904,20210918,2021-09-04,2021-09-18,saturday,11-09-2021,8733,GARE DE L'OUEST,WESTSTATION,POINT (146633.5 170956.4),None,NaN,11-09-2021,07:25:38,0.0,8733,8733,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161,saturday,1,M,0.0,0.0,26151.0,587.0,26738.000,07:25:38,318.000,0,NaN
1,112947202236269500,07:31:53,07:31:53,8742,2,0,0,NaN,BEEKKANT,NaN,50.853386,4.322974,NaN,NaN,0.0,1.0,2,236269500,STOCKEL,0,8912798,001m0080,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF,27141,15.0,cluster_0,15.0,0,0,0,0,0,0,1,0,20210904,20210918,2021-09-04,2021-09-18,saturday,11-09-2021,8742,BEEKKANT,BEEKKANT,POINT (146776.5 171444.3),POINT (146633.5 170956.4),508.424439,11-09-2021,07:32:21,0.0,8742,8742,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161,saturday,1,M,1.0,1.0,26308.0,833.0,27141.000,07:32:21,28.000,0,NaN
2,112947202236269500,07:33:33,07:33:33,8292,3,0,0,NaN,ETANGS NOIRS,NaN,50.857125,4.333143,NaN,NaN,0.0,12.0,2,236269500,STOCKEL,0,8912798,001m0080,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF,27233,15.0,cluster_0,15.0,0,0,0,0,0,0,1,0,20210904,20210918,2021-09-04,2021-09-18,saturday,11-09-2021,8292,ETANGS NOIRS,ZWARTE VIJVERS,POINT (147492.7 171859.9),POINT (146776.5 171444.3),828.049395,11-09-2021,07:33:53,0.0,8292,8292,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161,saturday,1,M,1.0,1.0,26370.0,863.0,27233.000,07:33:53,20.000,0,NaN
3,112947202236269500,07:34:44,07:34:44,8282,4,0,0,NaN,COMTE DE FLANDRE,NaN,50.854705,4.340542,NaN,NaN,0.0,13.0,2,236269500,STOCKEL,0,8912798,001m0080,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF,27295,15.0,cluster_0,15.0,0,0,0,0,0,0,1,0,20210904,20210918,2021-09-04,2021-09-18,saturday,11-09-2021,8282,COMTE DE FLANDRE,GRAAF VAN VLAAND.,POINT (148013.6 171590.4),POINT (147492.7 171859.9),586.487050,11-09-2021,07:34:55,0.0,8282,8282,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161,saturday,1,M,1.0,1.0,26432.0,863.0,27295.000,07:34:55,11.000,0,NaN
4,112947202236269500,07:36:08,07:36:08,8272,5,0,0,NaN,SAINTE-CATHERINE,NaN,50.851900,4.348012,NaN,NaN,0.0,14.0,2,236269500,STOCKEL,0,8912798,001m0080,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF,27385,15.0,cluster_0,15.0,0,0,0,0,0,0,1,0,20210904,20210918,2021-09-04,2021-09-18,saturday,11-09-2021,8272,SAINTE-CATHERINE,SINT-KATELIJNE,POINT (148539.5 171278.2),POINT (148013.6 171590.4),611.587811,11-09-2021,07:36:25,0.0,8272,8272,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161,saturday,1,M,1.0,1.0,26524.0,861.0,27385.000,07:36:25,17.000,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

## Exporting a work sample

In [5]:
res_g = res.groupby(by=["route_short_name", "trip_id", "service_id", "date_normalized"])

In [11]:
thresh = 100
g = []
i = 0
for name, group in tqdm(res_g):
    if i <= thresh:
        g.append(group)
    i += 1

sample = pd.concat(g)

100%|██████████| 314344/314344 [00:11<00:00, 27210.60it/s]


In [13]:
sample.to_pickle("data/computed/finally_matches_subsampled.pkl")

In [14]:
sample

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,route_id,service_id,trip_headsign,direction_id,block_id,shape_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,time_seconds,headway_min,clusters,cluster_agg_value,regularity,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,start_date_ft,end_date_ft,label,date_normalized,stop_id_clean_c,descr_fr,descr_nl,geometry,geom_position_before,dist_m,date,time,distance_from_point,stop_id__current,stop_id_cleaned,stop_id__terminus,stop_code__terminus,stop_name__terminus,stop_desc__terminus,stop_lat__terminus,stop_lon__terminus,zone_id__terminus,stop_url__terminus,location_type__terminus,parent_station__terminus,stop_id_cleaned__terminus,date_label,ligne_cleaned,mode,lag_-1,lag_+1,t_lag_before,t_diff(sec),adjusted_arrival_time(ts),adjusted_arrival_time,diff_time,vehicule_id,approx_rec
0,112947202236269500,07:30:56,07:30:56,8733,1,0,0,NaN,GARE DE L'OUEST,NaN,50.848999,4.320948,NaN,NaN,0.0,2.0,2,236269500,STOCKEL,0,8912798,001m0080,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF,26738,15.0,cluster_0,15.0,0,0,0,0,0,0,1,0,20210904,20210918,2021-09-04,2021-09-18,saturday,11-09-2021,8733,GARE DE L'OUEST,WESTSTATION,POINT (146633.5 170956.4),None,NaN,11-09-2021,07:25:38,0.0,8733,8733,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161,saturday,1,M,0.0,0.0,26151.0,587.0,26738.000000,07:25:38,318.0,0,NaN
1,112947202236269500,07:31:53,07:31:53,8742,2,0,0,NaN,BEEKKANT,NaN,50.853386,4.322974,NaN,NaN,0.0,1.0,2,236269500,STOCKEL,0,8912798,001m0080,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF,27141,15.0,cluster_0,15.0,0,0,0,0,0,0,1,0,20210904,20210918,2021-09-04,2021-09-18,saturday,11-09-2021,8742,BEEKKANT,BEEKKANT,POINT (146776.5 171444.3),POINT (146633.5 170956.4),508.424439,11-09-2021,07:32:21,0.0,8742,8742,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161,saturday,1,M,1.0,1.0,26308.0,833.0,27141.000000,07:32:21,28.0,0,NaN
2,112947202236269500,07:33:33,07:33:33,8292,3,0,0,NaN,ETANGS NOIRS,NaN,50.857125,4.333143,NaN,NaN,0.0,12.0,2,236269500,STOCKEL,0,8912798,001m0080,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF,27233,15.0,cluster_0,15.0,0,0,0,0,0,0,1,0,20210904,20210918,2021-09-04,2021-09-18,saturday,11-09-2021,8292,ETANGS NOIRS,ZWARTE VIJVERS,POINT (147492.7 171859.9),POINT (146776.5 171444.3),828.049395,11-09-2021,07:33:53,0.0,8292,8292,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161,saturday,1,M,1.0,1.0,26370.0,863.0,27233.000000,07:33:53,20.0,0,NaN
3,112947202236269500,07:34:44,07:34:44,8282,4,0,0,NaN,COMTE DE FLANDRE,NaN,50.854705,4.340542,NaN,NaN,0.0,13.0,2,236269500,STOCKEL,0,8912798,001m0080,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF,27295,15.0,cluster_0,15.0,0,0,0,0,0,0,1,0,20210904,20210918,2021-09-04,2021-09-18,saturday,11-09-2021,8282,COMTE DE FLANDRE,GRAAF VAN VLAAND.,POINT (148013.6 171590.4),POINT (147492.7 171859.9),586.487050,11-09-2021,07:34:55,0.0,8282,8282,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161,saturday,1,M,1.0,1.0,26432.0,863.0,27295.000000,07:34:55,11.0,0,NaN
4,112947202236269500,07:36:08,07:36:08,8272,5,0,0,NaN,SAINTE-CATHERINE,NaN,50.851900,4.348012,NaN,NaN,0.0,14.0,2,236269500,STOCKEL,0,8912798,001m0080,1,GARE DE L'OUEST - STOCKEL,NaN,1,NaN,C4008F,FFFFFF,27385,15.0,cluster_0,15.0,0,0,0,0,0,0,1,0,20210904,20210918,2021-09-04,2021-09-18,saturday,11-09-2021,8272,SAINTE-CATHERINE,SINT-KATELIJNE,POINT (148539.5 171278.2),POINT (148013.6 171590.4),611.587811,11-09-2021,07:36:25,0.0,8272,8272,8161,NaN,STOCKEL,NaN,50.841872,4.464541,NaN,NaN,0.0,29.0,8161,saturday,1,M,1.0,1.0,26524.0,861.0,27385.000000,07:36:25,17.0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...